In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense

In [9]:
train = pd.read_csv('./bike_data/train.csv', parse_dates=['datetime'])
test = pd.read_csv('./bike_data/test.csv', parse_dates=['datetime'])
sub = pd.read_csv('./bike_data/sampleSubmission.csv')

train.shape, test.shape, sub.shape

((10886, 12), (6493, 9), (6493, 2))

In [10]:
print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dt

In [11]:
train['hour'] = train['datetime'].dt.hour
test['hour'] = test['datetime'].dt.hour

train['dayofweek'] = train['datetime'].dt.dayofweek
test['dayofweek'] = test['datetime'].dt.dayofweek

In [12]:
input_col = ['hour', 'temp', 'dayofweek', 'workingday', 'season', 'weather']
labeled_col = ['count']

In [13]:
X = train[input_col]
y = train[labeled_col]
X_val = test[input_col]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
# 난수 발생 패턴 결정 0
seed = 0
np.random.seed(seed)

In [16]:
model = Sequential()
model.add(Dense(30, input_dim=len(input_col), activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                210       
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 15)                240       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 931
Trainable params: 931
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 491us/step - loss: 30530.5391
Epoch 2/20
817/817 [==============================] - 0s 497us/step - loss: 21812.2539
Epoch 3/20
817/817 [==============================] - 0s 495us/step - loss: 19686.7070
Epoch 4/20
817/817 [==============================] - 0s 484us/step - loss: 18817.6738
Epoch 5/20
817/817 [==============================] - 0s 470us/step - loss: 18563.1113
Epoch 6/20
817/817 [==============================] - 0s 486us/step - loss: 18378.8984
Epoch 7/20
817/817 [==============================] - 0s 486us/step - loss: 18184.6074
Epoch 8/20
817/817 [==============================] - 0s 515us/step - loss: 17970.4355
Epoch 9/20
817/817 [==============================] - 0s 486us/step - loss: 17764.5859
Epoch 10/20
817/817 [==============================] - 0s 467us/step - loss: 17498.4980
Epoch 11/20
817/817 [==============================] - 0s 473us/step - loss: 17169.3340
Epoch 12/20
817/817 [====================

In [19]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 492us/step - loss: 15174.9893


15174.9892578125

In [20]:
pred = model.predict(X_val)

203/203 [==============================] - 0s 405us/step


In [21]:
sub.columns

Index(['datetime', 'count'], dtype='object')

In [22]:
sub['count'] = pred
sub.loc[sub['count'] < 0, 'count'] = 0

In [24]:
sub.to_csv('./output/nn_sub_2207.csv', index=False)